In [22]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from typing import Optional, Any, List, Union, TypeVar, Type, cast, Callable


class Line:
    Title: str
    Letter: str
    def __init__(self, Letter, Title):
        self.Letter = Letter
        self.Title = Title   

class subParagraph:
    Title: str
    Lines: Optional[List[Line]]
    def __init__(self, Title, Lines):
        self.Title = Title   
        self.Lines = Lines
        
class Paragraph:
    Title: str
    Number: int
    subParagraphs: Optional[List[subParagraph]]
    def __init__(self, Number, Title, subParagraphs):
        self.Number = Number
        self.Title = Title
        self.subParagraphs = subParagraphs

class subSections:
    Title: str
    Number: int
    Paragraphs: Optional[List[Paragraph]]
    def __init__(self, Title, Number, Paragraphs):
        self.Title = Title
        self.Number = Number
        self.Paragraphs = Paragraphs

class Sections:
    Title: str
    Number: int
    subSections: Optional[List[subSections]]
    def __init__(self, Title, Number, subSections):
        self.Title = Title
        self.Number = Number
        self.subSections = subSections

def singlesection(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "xml")
        p5condition = False
        lists = False
        txt = ''
        pattern='Act '
        parts=[]
        partTitle=None
        itemNumber=None
        for part in soup.find_all('Part'):
            # Chapter 4, 6
            if (len(part.select('Pblock > P1group')) == 0):
                path = 'P1group'
            else:
                path = 'Pblock > P1group'
            for first in part.select(path):
                txt = txt + "<p>" + first.select('Pnumber')[0].text + ". " + first.find('Title').text+"</p>"
                titleSection= first.find('Title').text
                numberSection= first.select('Pnumber')[0].text

                p1 = first.findAll("P1")
                if p1[0].has_attr('DocumentURI'):
                    txt = txt + "<p>" + p1[0]['DocumentURI']+"</p>"
                    if first.select('P1 > P1para >Text'):
                        txt = txt + first.select('P1 > P1para >Text')[0].text+" </p>"

                    p1paras = first.select('P1para');
                    blocks = first.select('BlockAmendment > P1group > P1 > P1para');
                    topPara1s = len(p1paras) - len(blocks)
                    for p1para in p1paras[:topPara1s]:
                        if (p1para.find('P2') == None):
                            elements = []
                        elif (len(p1para.select('P5'))):
                            p5condition = True
                            elements = p1para.find_all_next()
                        else:
                            elements = p1para.find_all_next()
                        for element in elements:
                            subParts=[];
                            if (element.name == 'P2'):
                                if (element.has_attr('DocumentURI')):
                                    txt = txt + "  <p> " + element.select('Pnumber')[0].text+" </p>"
                                    txt = txt + "  <p> " + element.select('P2para > Text')[0].text+" </p>"
                                    partNumber = element.select('Pnumber')[0].text;
                                    partTitle  = element.select('P2para > Text')[0].text

                                    p2 = element;
                                    if (len(p2.select('P2 > P2para > BlockAmendment > P1group')) != 0 and not p5condition):
                                        for block in p2.select('P2 > P2para > BlockAmendment > P1group'):
                                            txt = txt + "  <p> " + block.select('P1 > Pnumber')[0].text+" </p>"
                                            txt = txt + "  <p> " + block.select('Title')[0].text+" </p>"

                                            textIndex = 0
                                            for p1 in block.select('P1 > P1para > P2'):
                                                txt = txt + " <p> " + p1.select('Pnumber')[0].text + " " + p1.find('Text').text+" </p>"

                                                # 185
                                                p3s = p1.select('P2 > P2para > P3')
                                                final_text = ""
                                                for p3 in p3s:
                                                    txt = txt + " <p> " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text+" </p>"
                                                    final_text = p3.find_next_sibling('Text')
                                                    for p4 in p3.select('P3 > P3para > P4'):
                                                        txt = txt + " <p> " + p4.select('Pnumber')[0].text + " " + p4.find('Text').text+" </p>"

                                                        array = block.select('P2 > P2para > Text')
                                                        del array[0]
                                                        for p2para in array:
                                                            txt = txt + "  <p> " + p2para.text+" </p>"

                                                if (type(final_text) is str):  # 185, 179
                                                    txt = txt + "  <p> " +final_text+" </p>"
                                                elif (hasattr(final_text, 'text')):
                                                    txt = txt + "  <p> " + final_text.text+" </p>"

                                    elif (len(p2.select('P5')) != 0):
                                        txt = txt + "<p>" + p5(p2) + " </p>"
                                    elif (len(p2.select(
                                            'P2 > P2para > BlockAmendment')) != 0):  # 211( Text -> P3), #5 (P2 -> P2Para),#185
                                        for block in p2.select('P2 > P2para > BlockAmendment'):
                                            textArray = block.select('Text')
                                            rootItems=[]
                                            if (len(textArray) > 0):    
                                                # validate this step
                                                for p3 in block.select('BlockAmendment > P3'):
                                                    txt = txt + " <p> " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text+" </p>"
                                                for p2 in block.select('BlockAmendment > P2'):
                                                    text_counter=0
                                                    txt = txt + "  <p> " + p2.select('Pnumber')[0].text + " " + p2.find('Text').text+" </p>"
                                                    subPartNumber=p2.select('Pnumber')[0].text;
                                                    subPartTitle=p2.find('Text').text;
                                                    text_counter=text_counter+1
                                                    items=[]
                                                    for p3 in p2.select('P2para > P3'):
                                                        txt = txt + " <p> " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text+" </p>"
                                                        items.append(Line(p3.select('Pnumber')[0].text, p3.find('Text').text))
                                                        text_counter=text_counter+1                                                        
                                                    ## for loop ?? - an example for 193
                                                    if(text_counter < len(p2.select('Text'))):
                                                        txt = txt + " <p> " + p2.select('Text')[text_counter].text + " </p>"
                                                        items.append(Line("", p2.select('Text')[text_counter].text))                                                    
                                                    subParts.append(Paragraph(subPartNumber,subPartTitle,[subParagraph(' ', items)]))

                                    elif (len(p2.select('P2 > P2para > P3')) != 0):
                                        if (p2.has_attr('DocumentURI')):
                                            for p3 in p2.select('P2 > P2para > P3'):
                                                txt = txt + " " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text + " </p>"
                                                subPartNumber = p3.select('Pnumber')[0].text;
                                                subPartTitle = p3.find('Text').text;  
                                                subParts.append(Paragraph(subPartNumber, subPartTitle,[]))
                                                for p4 in p3.select('P3 > P3para > P4'):
                                                    # PNUMBER and text display
                                                    txt = txt + " <p> " + p4.select('Pnumber')[0].text + " " + p4.find('Text').text+ " </p>"

                                            textArray = p3.select('P3 > P3para >Text')  # 5
                                            del textArray[0]
                                            for p3para in textArray:
                                                txt = txt + " <p> " + p3para.text+ " </p>"
                                                subParts.append(Paragraph("", p3para.text,[]))

                                            array = p2.select('P2 > P2para > Text')
                                            del array[0]
                                            for p2para in array:
                                                txt = txt+ " <p> " + p2para.text+ " </p>"
                                                subParts.append(Paragraph("", p2para.text,[]))

                                                
                                    
                                    if (len(p2.find_all('UnorderedList')) > 0):
                                        lists = True
                                    rootItems=[]
                                    for unorderedlist in p2.find_all('UnorderedList'):
                                        for listItem in unorderedlist.find_all('ListItem'):
                                            txt = txt + "  && <p> " + listItem.find('Para').text+ " </p>"
                                            titleSubPart = listItem.find('Para').text;
                                            items=[]
                                            for orderList in listItem.find_all('OrderedList'):
                                                for listItem1 in orderList.find_all('ListItem'):
                                                    if (listItem1.has_key('NumberOverride')):
                                                        txt = txt + "^^ <p> " + listItem1['NumberOverride'] + " </p>"
                                                        itemNumber=listItem1['NumberOverride'];
                                                    txt = txt + " ^^ <p> " + listItem1.find('Para').text + " </p>"
                                                    itemTitle=listItem1.find('Para').text;
                                                    items.append(Lines(itemNumber, itemTitle))
                                            rootItems.append(subParagraph(titleSubPart, items))
                                    if(len(rootItems)>0):
                                         subParts.append(Paragraph("","",rootItems))
                                if(partTitle!=None):            
                                    parts.append(subSections(partTitle, partNumber,subParts))
                        
                            elif (element.name == 'UnorderedList' and not p5condition and not lists):  # 98, dublication, 198
                                if (type(p1).__name__ == 'ResultSet'):
                                    path = p1[0]
                                else:
                                    path = p1
                                for unorderedlist in path.find_all('UnorderedList'):
                                    for listItem in unorderedlist.find_all('ListItem'):
                                        txt = txt + "  <p> " + listItem.find('Para').text + " </p>"
                                        for orderList in listItem.find_all('OrderedList'):
                                            for listItem1 in orderList.find_all('ListItem'):
                                                if (listItem1.has_key('NumberOverride')):
                                                    txt = txt + " <p> " + listItem1['NumberOverride']+ " </p>"
                                                txt = txt + "  <p> " + listItem1.find('Para').text+ " </p>"

                            elif (element.name == 'BlockText'):
                                for blockText in p1[0].select('BlockText > Para > Text'):
                                    txt = txt + blockText.text+ " </p>"

                if (first.select('P1para > P2') == None):
                    for P1para in first.select('P1para'):
                        txt = txt + " <p> " + P1para.find('Text').text+ " </p>"

                        # do we need this condition? # 270
                if (lists):
                    for unorderedlist in first.select('P1para > UnorderedList'):
                        for listItem in unorderedlist.find_all('ListItem'):
                            txt = txt + "  <p> " + listItem.find('Para').text+ " </p>"
                            for orderList in listItem.find_all('OrderedList'):
                                for listItem1 in orderList.find_all('ListItem'):
                                    if (listItem1.has_key('NumberOverride')):
                                        txt = txt + " <p> " + listItem1['NumberOverride']+ " </p>"
                                    txt = txt + "  <p> " + listItem1.find('Para').text+ " </p>"

                # last text - could require a for loop here # 99 case
                if (len(first.select('P1 > P1para > P2')) == 0):
                    x = len(first.find_all('Text')) - 1
                    txt = txt + " <p> " +first.find_all('Text')[x].text+ " </p>"

            return  Sections(titleSection, numberSection, parts)
           # return txt

        
def p3(block):
    return None

def p5(block):
    txt = " "
    txt = txt + block.select('Title')[0].text+" </p>"
    for p in block.select('P'):
        txt = txt + p.find('Text').text+" </p>"
        for unorderedlist in p.select('UnorderedList'):
            for listItem in unorderedlist.find_all('ListItem'):
                txt = txt + "  <p> " + listItem.find('Para').text+" </p>"
                for orderList in listItem.find_all('OrderedList'):
                    for listItem1 in orderList.find_all('ListItem'):
                        if (listItem1.has_key('NumberOverride')):
                            txt = txt + "  <p> " +listItem1['NumberOverride']+" </p>"
                        txt = txt + "  <p> " + listItem1.find('Para').text+" </p>"
        for p5 in p.select('P5'):
            txt = txt + p5.find('Pnumber').text+" </p>"
            for p5para in p5.select('P5para'):
                txt = txt + p5para.find('Text').text+" </p>"
    return txt

# no schedules captured
def exists (str, pattern):
    if pattern in str:
        print(str)

# report 77, 78
url="https://www.legislation.gov.uk/ukpga/2004/34/section/97/data.xml"

section=singlesection(url)

for x in section.subSections:
    print(x.Number+" "+x.Title)
    rootSection=x.Number
    for y in x.Paragraphs:
        print(y.Number+"  "+y.Title)
        for z in y.subParagraphs:
            print(y.Number+"  "+y.Title)
            for t in z.Lines:
                print(str(t.Letter)+"  "+str(t.Title))

1 This section applies in relation to orders made by residential property tribunals under section 96(5).
2 Where, on an application by the local housing authority, the tribunal is satisfied—
a  that a person has been convicted of an offence under section 95(1) in relation to the house, and
b  that—
  during any period during which it appears to the tribunal that such an offence was being committed in relation to the house,
  the tribunal must make a rent repayment order requiring the appropriate person to pay to the authority the amount mentioned in subsection (2A).
  This is subject to subsections (3), (4) and (8).
2A The amount referred to in subsection (2) is—
a  an amount equal to—
b  an amount equal to the total amount of housing benefit paid as mentioned in subsection (2)(b)(ii),
  (as the case may be).
3 If the total of the amounts received by the appropriate person in respect of periodical payments payable as mentioned in paragraph (b) of subsection (2)
					(“the rent total”) 